Building a recommendation system that predicts the binary outcome of whether the user will like the movie or not, by leveraging AutoEncoders. 

Datasets are taken from https://grouplens.org/datasets/movielens/


## Data Preprocessing

In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
# Importing the datasets
movies = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [3]:
movies.head(15)

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [4]:
movies.shape

(3883, 3)

In [5]:
# Importing the datasets
users = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [6]:
users.shape

(6040, 5)

In [7]:
users.head(10)

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


Description of Data:  
0 --> User ID  
1 --> Gender  
2 --> Age  
3 --> Occupation  
4 --> Zip Code  

In [8]:
# Importing the datasets
ratings = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings.shape

(1000209, 4)

In [9]:
ratings.head(10)

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


Descrption of the Data:  
0 --> Corresponds to the user ID   
1 --> Corresponds to the movie ID  
2 --> Ratings from 1-5  
3 --> Timestamps

In [10]:
# Preparing the training set and the test set
training_set = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ml-100k/u1.base', delimiter = '\t')

In [11]:
training_set.head()

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [12]:
training_set.shape

(79999, 4)

In [13]:
training_set = np.array(training_set, dtype = 'int')

In [14]:
test_set = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [15]:
# Getting the total no.of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_users

943

In [16]:
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
nb_movies

1682

In [17]:
# Converting he data into an array with users in lines and movies in columns
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1][data[:, 0] == id_users] 
    id_ratings = data[:, 2][data[:, 0] == id_users] 
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data

training_set = convert(training_set)

In [18]:
test_set = convert(test_set)

In [19]:
print(pd.DataFrame(training_set).head())

   0     1     2     3     4     5     ...  1676  1677  1678  1679  1680  1681
0   0.0   3.0   4.0   3.0   3.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
1   4.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
2   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
3   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
4   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0

[5 rows x 1682 columns]


In [20]:
print(pd.DataFrame(test_set).head())

   0     1     2     3     4     5     ...  1676  1677  1678  1679  1680  1681
0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
1   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
2   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
3   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
4   4.0   3.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0

[5 rows x 1682 columns]


In [21]:
# Converting the data into torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [22]:
training_set.shape

torch.Size([943, 1682])

## Creating the architecture of the Neural Network

In [23]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
        
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

## Training the SAE

In [24]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))

epoch: 1loss: tensor(1.7717)
epoch: 2loss: tensor(1.0965)
epoch: 3loss: tensor(1.0534)
epoch: 4loss: tensor(1.0382)
epoch: 5loss: tensor(1.0309)
epoch: 6loss: tensor(1.0265)
epoch: 7loss: tensor(1.0238)
epoch: 8loss: tensor(1.0219)
epoch: 9loss: tensor(1.0208)
epoch: 10loss: tensor(1.0195)
epoch: 11loss: tensor(1.0188)
epoch: 12loss: tensor(1.0185)
epoch: 13loss: tensor(1.0177)
epoch: 14loss: tensor(1.0175)
epoch: 15loss: tensor(1.0171)
epoch: 16loss: tensor(1.0172)
epoch: 17loss: tensor(1.0169)
epoch: 18loss: tensor(1.0165)
epoch: 19loss: tensor(1.0163)
epoch: 20loss: tensor(1.0161)
epoch: 21loss: tensor(1.0162)
epoch: 22loss: tensor(1.0160)
epoch: 23loss: tensor(1.0157)
epoch: 24loss: tensor(1.0158)
epoch: 25loss: tensor(1.0156)
epoch: 26loss: tensor(1.0155)
epoch: 27loss: tensor(1.0154)
epoch: 28loss: tensor(1.0154)
epoch: 29loss: tensor(1.0126)
epoch: 30loss: tensor(1.0117)
epoch: 31loss: tensor(1.0094)
epoch: 32loss: tensor(1.0092)
epoch: 33loss: tensor(1.0043)
epoch: 34loss: tens

## Testing the Neural Network

In [26]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('loss: '+ str(test_loss/s))

loss: tensor(0.9535)
